In [ ]:
# Rader Example 1pt3
# Written for Pyomo by CDR Jay Foraker

from __future__ import division
from pyomo.environ import *
# import necessary packages for Excel interface
import xlwings as xw

##### LOAD DATA FROM EXCEL #####
# Load data workbook
# Full path needed!
wb = xw.Book(r'C:\\Output\\xlwings_Pyomo_test.xlsm')
sinput = wb.sheets['Input']
soutput = wb.sheets['Output']

model = ConcreteModel()

## Define sets ##
model.I = Set(initialize=[1,2,3,4], doc='Set of Students')
model.J = Set(initialize=[1,2,3,4,5,6], doc='Set of Tasks')

## Define parameters ##

# param time :   1   2   3   4   5   6 :=
# 1              12  5   8   9   6   11
# 2              14  8   7   11  10  5
# 3              10  9   9   8   7   8
# 4              11  8   10  10  9   10;

timetab = {}

for i in model.I:
    timetab[i,1] = sinput.range('C' + str(i + 2)).value
    timetab[i,2] = sinput.range('D' + str(i + 2)).value
    timetab[i,3] = sinput.range('E' + str(i + 2)).value
    timetab[i,4] = sinput.range('F' + str(i + 2)).value
    timetab[i,5] = sinput.range('G' + str(i + 2)).value
    timetab[i,6] = sinput.range('H' + str(i + 2)).value

model.time = Param(model.I, model.J, initialize=timetab, doc='time student i would need to complete task j in hours')

# decision variables
model.x = Var(model.I, model.J, within=Binary, doc='1 if student i is assigned to task j, 0 otherwise')

# objective function: NOTE that default sense is minimize in Pyomo
# Minimize the total amount of time required to complete all tasks
def cost_rule(model):
    return sum(model.time[i,j]*model.x[i,j] for i in model.I for j in model.J)
model.cost = Objective(rule=cost_rule)

# constraints
def student_min_rule(model, i):
    return sum(model.x[i,j] for j in model.J) >= 1
model.min_rule = Constraint(model.I, rule=student_min_rule, doc='Each student has at least one task')

def student_max_rule(model, i):
    return sum(model.x[i,j] for j in model.J) <= 2
model.max_rule = Constraint(model.I, rule=student_max_rule, doc='Each student has at most two tasks')

def one_student_per_task(model, j):
    return sum(model.x[i,j] for i in model.I) >= 1
model.one_student_rule = Constraint(model.J, rule=one_student_per_task, doc='At least one student to each task')

# display output
def pyomo_postprocess(options=None, instance=None, results=None):
    # to display value and information about all decision variables
    #model.x.display()
    # to display a pretty printed version of the model structure
    #model.pprint()
    print("Results\n")
    print("Minimum Total Time is " + str(value(model.cost)) +"\n")
    print("Student   Task")
    print("-------   ----")
    for i in model.I:
        for j in model.J:
            if model.x[i,j].value > 0.5:
                print("      " + str(i) + "   " + "   " + str(j))
    #Now test output functions using XLWings
    # Clear current output in worksheet
    wb.sheets['Output'].range('A2').clear()
    wb.sheets['Output'].range('A7:B30').clear()
    # Output objective value to Excel
    soutput.range('A2').value = value(model.cost)
    # Output solution to Excel
    counter = 0
    for i in model.I:
        for j in model.J:
            if model.x[i,j].value > 0.5:
                soutput.range('A' + str(counter + 7)).value = i
                soutput.range('B' + str(counter + 7)).value = j
                counter = counter + 1
    
# This is an optional code path that allows the script to be run outside of
# pyomo command-line.  For example:  python transport.py
if __name__ == '__main__':
    # This emulates what the pyomo command-line tools does
    from pyomo.opt import SolverFactory
    import pyomo.environ
    opt = SolverFactory("glpk")
    results = opt.solve(model)
    #sends results to stdout
    results.write()
    print("\nDisplaying Solution\n" + '-'*60)
    pyomo_postprocess(None, None, None)